# JOINS in SQL

## Retail Dataset Tables

This retail dataset is organized across the following tables:

- **customers**: Stores customer profile and location details
- **orders**: One record per order placed by a customer, including order date and status
- **order_items**: Line-level product details within each order (revenue is calculated here)
- **products**: Product catalog information
- **categories**: Product categorization
- **departments**: High-level product grouping

Refer to the schema diagram below for table relationships:

![ERD Placeholder](../images/ecommerce_erd.png)

Retrieve order-level and item-level information together so that individual products sold within each order can be analyzed along with the order date.

### INNER JOIN 

- Orders with Their Line Items

- This query combines `orders` and `order_items` to return only those records where an order has at least one corresponding order item.

- Each row represents a product sold as part of an order
- Orders without items are excluded
- Used for sales and revenue analysis

In [ ]:
SELECT o.order_date,
       oi.order_item_product_id,
       oi.order_item_subtotal
FROM orders AS o
INNER JOIN order_items AS oi
    ON o.order_id = oi.order_item_order_id;

### LEFT OUTER JOIN – Orders With or Without Items

- Identify all orders, including those that may not yet have any associated order items. This query returns all orders, even if they do not yet have entries in `order_items`.
- Ensures no order is missed
- Orders without items appear with NULL item fields
- Useful for operational monitoring and data quality checks

In [ ]:
SELECT o.order_id,
       o.order_date,
       oi.order_item_id,
       oi.order_item_product_id,
       oi.order_item_subtotal
FROM orders AS o
LEFT JOIN order_items AS oi
    ON o.order_id = oi.order_item_order_id
ORDER BY o.order_id;

### JOIN with Filtering and Aggregation

Calculate daily product-level revenue, but only for orders that are finalized (status is COMPLETE or CLOSED).

This query joins orders with order items, filters only finalized orders, and aggregates revenue at the order-date and product level.

- Revenue is calculated from `order_items`
- Only finalized orders contribute to revenue
- Results support daily sales reporting

In [ ]:
SELECT o.order_date,
       oi.order_item_product_id,
       ROUND(SUM(oi.order_item_subtotal), 2) AS order_revenue
FROM orders AS o
INNER JOIN order_items AS oi
    ON o.order_id = oi.order_item_order_id
WHERE o.order_status IN ('COMPLETE', 'CLOSED')
GROUP BY o.order_date, oi.order_item_product_id
ORDER BY o.order_date, order_revenue DESC;